In [1]:
import sys
import subprocess
import json
import requests

In [2]:
ORION="http://fiware-orion:1026/"
QUANTUMLEAP="http://fiware-quantumleap:8668/"
IOT_AGENT="http://fiware-iotagent:4041/"
HEADER_JSON={"Content-Type": "application/json"}
HEADER_TEXT_PLAIN={"Content-Type": "text/plain"}
true=True
false=False

In [4]:
a=requests.get(QUANTUMLEAP+"v2/version")
a.text

'{\n  "version": "0.7.5"\n}\n'

In [83]:
r=requests.get(ORION+"/v2/entities",params={"type":"House"})
print(len(r.json()))

0


In [33]:
data={
    "type": "House",
    "id": "urn:ngsi-ld:Store:002",
    "address": {
        "type": "PostalAddress",
        "value": {
            "streetAddress": "Friedrichstraße 44",
            "addressRegion": "Berlin",
            "addressLocality": "Kreuzberg",
            "postalCode": "10969"
        },
        "metadata": {
            "verified": {
                "value": true,
                "type": "Boolean"
            }
        }
    },
    "location": {
        "type": "geo:json",
        "value": {
             "type": "Point",
             "coordinates": [13.3903, 52.5075]
        }
    },
    "name": {
        "type": "Text",
        "value": "Checkpoint Markt"
    }
}

In [58]:
h={"Content-Type": "application/json","fiware-service": "openiot","fiware-servicepath":"/"}
r=requests.post(ORION+"v2/entities",headers=h,data=json.dumps(data))
print(r.text)

{"error":"Unprocessable","description":"Already Exists"}


In [7]:
h={"fiware-service": "iota001","fiware-servicepath":"/"}
r=requests.get(ORION+'v2/entities',headers=h).json()
for i in r:
    print(i)

{u'timezone': {u'type': u'Text', u'value': u'Asia/Taipei', u'metadata': {}}, u'type': u'House', u'id': u'urn:ngsi-ld:House:001', u'name': {u'type': u'Text', u'value': u'Rteslab', u'metadata': {}}, u'address': {u'type': u'PostalAddress', u'value': {u'addressRegion': u'Tainan City', u'streetAddress': u'No.1, University Road', u'Country': u'Taiwan,R.O.C.'}, u'metadata': {}}}
{u'refHouse': {u'type': u'Relationship', u'value': u'urn:ngsi-ld:House:001', u'metadata': {}}, u'type': u'Floor', u'id': u'urn:ngsi-ld:Floor:1', u'floor': {u'type': u'Text', u'value': u'1F', u'metadata': {}}}
{u'refHouse': {u'type': u'Relationship', u'value': u'urn:ngsi-ld:House:001', u'metadata': {}}, u'type': u'Floor', u'id': u'urn:ngsi-ld:Floor:2', u'floor': {u'type': u'Text', u'value': u'2F', u'metadata': {}}}
{u'refHouse': {u'type': u'Relationship', u'value': u'urn:ngsi-ld:House:001', u'metadata': {}}, u'type': u'Floor', u'id': u'urn:ngsi-ld:Floor:3', u'floor': {u'type': u'Text', u'value': u'B1', u'metadata': {}}

In [82]:
if type(r)==list:
    id_list=[]
    for i in r:
        id_list.append(i['id'])
        print(i['id'])

In [80]:
h2={"fiware-service": "iota001","fiware-servicepath":"/"}
for node_name in id_list:
    r=requests.delete(ORION+"v2/entities/"+node_name,headers=h2)
    print(r.text)

In [158]:
curl -iX POST \
  'http://fiware_iotagent:4041/iot/devices' \
  -H 'Content-Type: application/json' \
  -H 'fiware-service: openiot' \
  -H 'fiware-servicepath: /' \
  -d '{
 "devices": [
   {
     "device_id":   "motion003",
     "entity_name": "urn:ngsi-ld:Motion:003",
     "entity_type": "Motion",
     "timezone":    "Europe/Berlin",
     "attributes": [
       { "object_id": "c", "name": "count", "type": "Integer" }
     ],
     "static_attributes": [
       { "name":"refStore", "type": "Relationship", "value": "urn:ngsi-ld:Store:001"},
       {"name":"timeStamp","type":"String","value":"123456"}
     ]
   }
 ]
}
'

In [ ]:
curl -X GET \
  'http://fiware-iotagent:4041/iot/devices' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /'

In [ ]:
curl -X GET \
  'http://fiware-orion:1026/v2/entities' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /'

In [ ]:
curl -iX POST \
  'http://fiware-iotagent:4041/iot/devices' \
  -H 'Content-Type: application/json' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /' \
  -d '{
  "devices": [
    {
      "device_id": "Sensor001",
      "entity_name": "urn:ngsi-ld:Sensor:001",
      "entity_type": "DummySensor",
      "attributes": [
       {"object_id": "COUNT", "name": "count", "type": "Float"}
      ],
      "static_attributes": [
        {"name":"sensorName", "type": "Text", "value": "Tempempure Sensor"},
        {"name":"fieldName", "type": "Text", "value": "In Room Tempempure"},
        {"name":"refRoom", "type": "Relationship", "value": "urn:ngsi-ld:Room:001"},
        {"name":"timeResolution", "type": "Integer", "value": "3600"},
        {"name":"sensorType", "type": "Text", "value": "Tempempure"},   
        {"name":"unit", "type": "Text", "value": "Celsius"},
        {"name":"predictionValue", "type": "Float", "value": "None"},
        {"name":"anomalyScore", "type": "Float", "value": "None"},
        {"name":"anomalyLikehood", "type": "Float", "value": "None"}
      ]
    }
  ]
}'

In [ ]:
curl -iX POST \
  'http://fiware-iotagent:4041/iot/services' \
  -H 'Content-Type: application/json' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /' \
  -d '{
 "services": [
   {
     "apikey":      "4jggokgpepnvsb2uv4s40d59ov",
     "cbroker":     "http://fiware-orion:1026",
     "entity_type": "Sensor",
     "resource":    "/iot/d"
   }
 ]
}'

In [ ]:
curl -X GET \
  'http://fiware-iotagent:4041/iot/devices' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /'

In [ ]:
curl -iX DELETE \
  'http://fiware-iotagent:4041/iot/devices/Sensor01' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /'

In [ ]:
curl -iX DELETE \
  'http://fiware-orion:1026/v2/entities/urn:ngsi-ld:Electricity:01' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /'

In [ ]:
curl -iX DELETE \
  'http://fiware-iotagent:4041/iot/services/?resource=/iot/d&apikey=4jggokgpepnvsb2uv4s40d59ov' \
  -H 'fiware-service: iota' \
  -H 'fiware-servicepath: /'

In [ ]:
curl -iX DELETE \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /' \
  --url 'http://fiware-orion:1026/v2/entities/urn:ngsi-ld::001'

In [ ]:
curl -iX POST \
  'http://fiware-orion:1026/v2/subscriptions/' \
  -H 'Content-Type: application/json' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /' \
  -d '{
  "description": "Notify QuantumLeap of count changes of any Motion Sensor",
  "subject": {
    "entities": [
      {
        "idPattern": ".*",
        "type":"Sensor"
      }
    ],
    "condition": {
      "attrs": [
        "value"
      ]
    }
  },
  "notification": {
    "http": {
      "url": "http://flask-test:5000/"
    },
    "attrs": [
      "value"
    ],
    "metadata": ["dateCreated", "dateModified"]
  },
  "throttling": 1
}'

In [ ]:
curl -X GET \
  'http://fiware-orion:1026/v2/subscriptions/' \
  -H 'fiware-service: iota001' \
  -H 'fiware-servicepath: /'

In [ ]:
curl -X GET \
  'http://fiware-quantumleap:8668/v2/entities/Sensor001/attrs/value?limit=3' \
  -H 'Accept: application/json' \
  -H 'Fiware-Service: openiot' \
  -H 'Fiware-ServicePath: /'

In [ ]:
curl -iX POST \
  'http://fiware-iotagent:7896/iot/d?k=4jggokgpepnvsb2uv4s40d59ov&i=Sensor001' \
  -H 'Content-Type: text/plain' \
  -d 'VALUE|4'

In [ ]:
curl -X POST \
'http://fiware-quantumleap:8668/v2/subscribe?orionUrl=http://fiware-orion:1026/v2&quantumleapUrl=http://fiware-quantumleap:8668/v2&entityType=Sensor' \
-H 'Accept: application/json' \
  -H 'Fiware-Service: openiot' \
  -H 'Fiware-ServicePath: /'

In [13]:
subprocess.run(["curl","-iX","POST","http://flask-test:5000","-d","{'ddd','abc'}"])

CompletedProcess(args=['curl', '-iX', 'POST', 'http://flask-test:5000', '-d', "{'ddd','abc'}"], returncode=0)

In [4]:

a={1,2,3,4,5,6}
print(type(a))

<class 'set'>
